### This notebook details the construction of a sample l2t database for full pipeline testing

In [1]:
import pandas as pd
import numpy as np

import duckdb

In [2]:
con = duckdb.connect('/mnt/s/PairProphet/Pairprophet.db')

In [7]:
# Make sample protein pairs table
cmd = """CREATE TEMP TABLE samp_protein_pairs AS
         SELECT * FROM protein_pairs LIMIT 50000"""

con.execute(cmd)

In [ ]:
# Make sample proteins table
cmd = """CREATE TEMP TABLE samp_proteins AS
         SELECT * FROM proteins
         WHERE proteins.pid IN 
         (SELECT DISTINCT meso_pid FROM samp_protein_pairs)
         OR proteins.pid IN
         (SELECT DISTINCT thermo_pid FROM samp_protein_pairs)"""

con.execute(cmd)

In [8]:
# Make sample taxa table
cmd = """CREATE TEMP TABLE samp_taxa AS
         SELECT * FROM taxa
         WHERE taxa.taxid IN 
         (SELECT DISTINCT meso_taxid FROM samp_protein_pairs)
         OR taxa.taxid IN
         (SELECT DISTINCT thermo_taxid FROM samp_protein_pairs)"""

con.execute(cmd)

In [9]:
# Make sample taxa_pairs table
cmd = """CREATE TEMP TABLE samp_taxa_pairs AS
         SELECT * FROM taxa_pairs
         WHERE taxa_pairs.query_id IN 
         (SELECT DISTINCT taxid FROM samp_taxa)
         AND taxa_pairs.subject_id IN 
         (SELECT DISTINCT taxid FROM samp_taxa)"""

con.execute(cmd)

In [ ]:
# Make sample taxa_lab table
cmd = """CREATE TEMP TABLE samp_taxa_pairs_lab AS
         SELECT * FROM taxa_pairs_lab
         WHERE taxa_pairs_lab.__index_level_0__ IN 
         (SELECT __index_level_0__ FROM samp_taxa_pairs)
      """

con.execute(cmd)

In [11]:
# Grab new tables as df and close connection to large database
samp_protein_pairs = con.execute("""SELECT * FROM samp_protein_pairs""").df()
samp_proteins = con.execute("""SELECT * FROM samp_proteins""").df()
samp_taxa = con.execute("""SELECT * FROM samp_taxa""").df()
samp_taxa_pairs = con.execute("""SELECT * FROM samp_taxa_pairs""").df()
samp_taxa_pairs_lab = con.execute("""SELECT * FROM samp_taxa_pairs_lab""").df()
con.close()

In [15]:
# Connect to new sample database and create tables
con = duckdb.connect('/mnt/s/PairProphet/l2t_50k.db')
con.execute("""CREATE OR REPLACE TABLE protein_pairs AS SELECT * FROM samp_protein_pairs""")
con.execute("""CREATE OR REPLACE TABLE proteins AS SELECT * FROM samp_proteins""")
con.execute("""CREATE OR REPLACE TABLE taxa AS SELECT * FROM samp_taxa""")
con.execute("""CREATE OR REPLACE TABLE taxa_pairs AS SELECT * FROM samp_taxa_pairs""")
con.execute("""CREATE OR REPLACE TABLE taxa_pairs_lab AS SELECT * FROM samp_taxa_pairs_lab""")

In [17]:
con.execute("""SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'""").df()

ConnectionException: Connection Error: Connection has already been closed